In [1]:
"""

10k_pycistopic_modelling.ipynb

This script is used to create the pycistopic object

authors: Roy Oelen

"""

'\n\n10k_pycistopic_modelling.ipynb\n\nThis script is used to create the pycistopic object\n\nauthors: Roy Oelen\n\n'

In [2]:
# imports
from pycisTopic.cistopic_class import CistopicObject
from scipy.sparse import csr_matrix
from scipy.io import mmread
import gzip
from sklearn.preprocessing import binarize
import pandas as pd
import os
from pycisTopic.lda_models import run_cgs_models_mallet
from numpy.random import choice
from pycisTopic.lda_models import evaluate_models
import pickle
import numpy as np
import glob
import re
from pycisTopic.topic_binarization import binarize_topics
from pycisTopic.topic_qc import compute_topic_metrics, plot_topic_qc, topic_annotation
import matplotlib.pyplot as plt
from pycisTopic.utils import fig2img
from pycisTopic.clust_vis import (
    find_clusters,
    run_umap,
    run_tsne,
    plot_metadata,
    plot_topic,
    cell_topic_heatmap
)


/home/umcg-roelen/miniconda3/envs/pycistopic_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-14 19:09:55,122	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
#######################################
# read the openness to nucleus matrix #
#######################################

# location of matrix
fragment_matrix_loc = '/groups/umcg-franke-scrna/tmp04/external_datasets/10x_multiome_10k_pbmcs/pycistopic/deconstructed/matrix.mtx.gz'
with open(fragment_matrix_loc, 'rb') as f:
    coo_fragment_matrix = mmread(fragment_matrix_loc)

# convert to csr format
fragment_matrix = csr_matrix(coo_fragment_matrix)


In [4]:
# locations
barcodes_loc = '/groups/umcg-franke-scrna/tmp04/external_datasets/10x_multiome_10k_pbmcs/pycistopic/deconstructed/barcodes.tsv.gz'
features_loc = '/groups/umcg-franke-scrna/tmp04/external_datasets/10x_multiome_10k_pbmcs/pycistopic/deconstructed/features.tsv.gz'

# load region names
region_names_file = gzip.open(features_loc, 'rb')
region_names = pd.read_csv(region_names_file, sep='\t', header=None).iloc[:,0].to_list()
region_names = [i.replace('-', ':', 1) for i in region_names]

# Read cell names
cell_names_file = gzip.open(barcodes_loc, 'rb')
cell_names = pd.read_csv(cell_names_file, sep='\t', header=None).iloc[:,0].to_list()


In [14]:
#########################################
# set up path to all the fragment files #
#########################################

# add the path
fragment_files = ['/groups/umcg-franke-scrna/tmp04/external_datasets/10x_multiome_10k_pbmcs/rounding/10k_PBMC_Multiome_nextgem_Chromium_X_rounded_atac_fragments.tsv.gz']


In [15]:
#############################
# read the nucleus metadata #
#############################

# location of the metadata for ATAC
atac_cell_metadata_loc = '/groups/umcg-franke-scrna/tmp04/external_datasets/10x_multiome_10k_pbmcs/pycistopic/deconstructed/metadata.tsv.gz'
atac_cell_metadata = pd.read_csv(atac_cell_metadata_loc, sep = '\t', low_memory = False)
# location of the metadata for RNA
rna_cell_metadata_loc = '/groups/umcg-franke-scrna/tmp04/external_datasets/10x_multiome_10k_pbmcs/seurat/10k_seurat_annotated_metadata.tsv.gz'
rna_cell_metadata = pd.read_csv(rna_cell_metadata_loc, sep = '\t', low_memory = False)
# merge what we find interesting
cell_metadata = atac_cell_metadata.merge(rna_cell_metadata[['barcode_1', 
                                                            'nCount_RNA', 
                                                            'nFeature_RNA', 
                                                            'nCount_SCT', 
                                                            'nFeature_SCT', 
                                                            'SCT_snn_res.1.2', 
                                                            'predicted.celltype.azi.l1', 
                                                            'predicted.celltype.azi.l1.score', 
                                                            'predicted.celltype.azi.l2', 
                                                            'predicted.celltype.azi.l2.score', 
                                                            'predicted.celltype.azi.l3', 
                                                            'predicted.celltype.azi.l3.score']], how = 'left', left_on = 'bc', right_on = 'barcode_1')
# Subset cell metadata 
cell_metadata = cell_metadata[cell_metadata.bc.isin(cell_names)]
# Filter cell names
cell_names = cell_metadata.bc.to_list()
# Names as rownames 
cell_metadata.index = cell_metadata.bc


In [17]:
#####################################
# binarize the accessibility matrix #
#####################################

# binarization of accessibility matrix
fragment_matrix_binarized = binarize(fragment_matrix, threshold=0)


In [21]:
##########################################
# create metadata for all of the regions #
##########################################

# location of the cpeaks annotation
cpeaks_annotation_loc = '/groups/umcg-franke-scrna/tmp04/external_datasets/cPeaks/cPeaks_info.tsv'
# read the annotations
cpeaks_annotation = pd.read_csv(cpeaks_annotation_loc, sep = ' ', low_memory = False)
# add a name column
cpeaks_annotation['name'] = cpeaks_annotation['chr_hg38'].astype(str) + ':' + cpeaks_annotation['start_hg38'].astype(str) + '-' + cpeaks_annotation['end_hg38'].astype(str)

# Subset region metadata metadata 
cpeaks_annotation = cpeaks_annotation[cpeaks_annotation.name.isin(region_names)]
# Names as rownames 
cpeaks_annotation.index = cpeaks_annotation.name


In [23]:
#############################
# create pycistopic object #
#############################

# construct object
cistopic_obj = CistopicObject(
    fragment_matrix=fragment_matrix,
    binary_matrix=fragment_matrix_binarized,
    cell_names=cell_names,
    region_names=region_names,
    region_data=cpeaks_annotation,
    cell_data=cell_metadata,
    path_to_fragments=fragment_files,
    project='10x_10k'
)


In [24]:
##########################
# save pycistopic object #
##########################

# location to store the object
pycistopic_object_loc = '/groups/umcg-franke-scrna/tmp04/external_datasets/10x_multiome_10k_pbmcs/pycistopic/reconstructed/10k_initial_pycistopic_object.pkl'

# save the object
with open(pycistopic_object_loc, 'wb') as f:
   pickle.dump(cistopic_obj, f)


In [3]:
##########################
# read pycistopic object #
##########################

# location to store the object
pycistopic_object_loc = '/groups/umcg-franke-scrna/tmp04/external_datasets/10x_multiome_10k_pbmcs/pycistopic/reconstructed/10k_initial_pycistopic_object.pkl'

# save the object
with open(pycistopic_object_loc, 'rb') as f:
   cistopic_obj = pickle.load(f)
    

In [4]:
#################################
# set paths for topic modelling #
#################################

# LDA will use Mallet, set the memory to be large enough
os.environ['MALLET_MEMORY'] = '100G'

# path to the Mallet binary
mallet_path = '/groups/umcg-franke-scrna/tmp04/software/Mallet-202108/bin/mallet'

# set temp directory
tmp_directory = '/groups/umcg-franke-scrna/tmp04/external_datasets/10x_multiome_10k_pbmcs/pycistopic/tmp_space/'

# set where we save the pickles
models_save_loc = '/groups/umcg-franke-scrna/tmp04/external_datasets/10x_multiome_10k_pbmcs/pycistopic/models/'


In [ ]:
####################################
# run LDA on the pycistopic object #
####################################

# get the models
models = run_cgs_models_mallet(
    cistopic_obj,
    n_topics = [5, 10, 20, 30, 40],
    n_cpu = 8,
    n_iter = 500,
    random_state = 7777,
    alpha = 50,
    alpha_by_topic = True,
    eta = 0.1,
    eta_by_topic = False,
    tmp_path = tmp_directory,
    save_path = models_save_loc,
    mallet_path = mallet_path,
)


2025-05-14 19:10:08,666 cisTopic     INFO     Formatting input to corpus
2025-05-14 19:10:09,389 cisTopic     INFO     Running model with 5 topics
2025-05-14 19:10:09,390 LDAMalletWrapper INFO     Serializing temporary corpus to /groups/umcg-franke-scrna/tmp04/external_datasets/10x_multiome_10k_pbmcs/pycistopic/tmp_space/corpus.txt
2025-05-14 19:10:34,299 LDAMalletWrapper INFO     Converting temporary corpus to MALLET format with: /groups/umcg-franke-scrna/tmp04/software/Mallet-202108/bin/mallet import-file --preserve-case --keep-sequence --token-regex \S+ --input /groups/umcg-franke-scrna/tmp04/external_datasets/10x_multiome_10k_pbmcs/pycistopic/tmp_space/corpus.txt --output /groups/umcg-franke-scrna/tmp04/external_datasets/10x_multiome_10k_pbmcs/pycistopic/tmp_space/corpus.mallet


In [ ]:
################
# select model #
################

# select 20 topics
model = evaluate_models(
    models,
    return_model = True,
    select_model = 20
)
